<a href="https://colab.research.google.com/github/kokolight/MetacriticProject/blob/main/DataScienceProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DataScience Project


## Creating the data set - Crawling 


### Main problem -
* Gaming companies develop games for years and cannot predict if their game will be sucessfull or not.

### Data -
Metacritic -

Description: Games ranking and reviews

Website: https://www.metacritic.com/

### Web scraping

### Data we scrape:
- Name - V
- Release Date - V
- Platforms - 
- Metascore - V
- Userscore
- Critic reviews
- Number of players
- Developer
- Publisher
- Maturaty Rating
- Genres
- Number of users ratings
- Number of critics ratings
- Number of awards
- Resolution

Possible:

- Number of positive/mixed/negative ratings
- Sound
- Online Playable (yes/no) 

In [104]:
headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.93 Safari/537.36'}
game_req = requests.get("https://www.metacritic.com/game/xbox-one/avicii-invector",headers=headers)
soup = BeautifulSoup(game_req.text)
soup.select('div.feature_userscore')[0].select('.metascore_w.user.large.game')
#soup.find("li", class_="summary_detail product_platforms").find_all("a")

#soup.find("div", class_="metascore_w user")

[<div class="metascore_w user large game positive">7.7</div>]

In [1]:
import requests
import bs4
from bs4 import BeautifulSoup
import pandas as pd
import threading

In [116]:
base_url = "https://www.metacritic.com"
browse_url = base_url + '/browse/games/score/metascore/all/all/filtered?'

month_dict = {'January':'1', 'February':'2', 'March':'3', 'April':'4', 'May': '5', 'June':'6', 'July':'7',
                    'August':'8', 'September':'9', 'October':'10', 'November':'11', 'December':'12'}

soup = BeautifulSoup('html.parser')

def metacritic_games_data(start_page, end_page):
  headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.93 Safari/537.36'}

  names = []
  metascores = []
  userscores = []
  num_of_platforms = []
  release_dates = []
  platforms = []
  hrefs = []

  for i in range(start_page, end_page):
    print(f"i = {i}")
    page_url = browse_url+'page={0}'.format(i)
    req = requests.get(page_url, headers=headers)
    soup.append(bs4.BeautifulSoup(req.text))

  games_records = soup.find_all("td", class_="clamp-summary-wrap")

  for game in games_records:
    title_obj = game.find('a', class_="title")
    title =title_obj.text
    href = title_obj.get("href")
    meta_score = game.find('div', class_="metascore_w").text
    platform = game.find("span", class_="data").text.strip()
    names.append(title)
    hrefs.append(href)
    metascores.append(meta_score)
    platforms.append(platform)
    date=game.find("div", class_="clamp-details").find_all("span")[2].text.replace(",", "")
    date_splitted = date.split(" ")
    release_date="{day}-{month}-{year}".format(day = date_splitted[1], month = month_dict[date_splitted[0]] , year = date_splitted[2])
    release_dates.append(release_date)
    
    # in game section
    game_url = base_url+''+href
    game_req = requests.get(game_url, headers=headers)
    print(f"URL: {game_url}")
    game_soup = BeautifulSoup(game_req.text, 'html.parser')
    user_score = game_soup.select('div.feature_userscore')[0].select('.metascore_w.user.large.game')[0].text
    #platforms_count = len(game_soup.find("li", class_="summary_detail product_platforms").find_all("a"))
    userscores.append(user_score)
    #num_of_platforms.append(platforms_count)

  df = pd.DataFrame({'name': names, 'metascore': metascores, 'userscore': userscores, 'release_date': release_dates, 
                     'platform': platforms})
  df.to_csv("./video_games_ratings.csv",  mode='a')
  return df

df = metacritic_games_data(1,2)
df


i = 1
URL: https://www.metacritic.com/game/xbox/burnout-3-takedown
URL: https://www.metacritic.com/game/pc/diablo
URL: https://www.metacritic.com/game/playstation-2/metal-gear-solid-3-subsistence
URL: https://www.metacritic.com/game/xbox-360/call-of-duty-modern-warfare-2
URL: https://www.metacritic.com/game/playstation-3/metal-gear-solid-4-guns-of-the-patriots
URL: https://www.metacritic.com/game/playstation-2/ncaa-football-2004
URL: https://www.metacritic.com/game/playstation-2/god-of-war
URL: https://www.metacritic.com/game/xbox/star-wars-knights-of-the-old-republic
URL: https://www.metacritic.com/game/pc/sid-meiers-civilization-iv
URL: https://www.metacritic.com/game/gamecube/madden-nfl-2004
URL: https://www.metacritic.com/game/playstation-2/virtua-fighter-4
URL: https://www.metacritic.com/game/nintendo-64/super-mario-64
URL: https://www.metacritic.com/game/pc/the-witcher-3-wild-hunt
URL: https://www.metacritic.com/game/playstation-2/burnout-3-takedown
URL: https://www.metacritic.co

,name,metascore,userscore,release_date,platform
0,Burnout 3: Takedown,94,7.7,7-9-2004,Xbox
1,Diablo,94,8.7,31-12-1996,PC
2,Metal Gear Solid 3: Subsistence,94,9.1,14-3-2006,PlayStation 2
3,Call of Duty: Modern Warfare 2,94,6.5,10-11-2009,Xbox 360
4,Metal Gear Solid 4: Guns of the Patriots,94,8.8,12-6-2008,PlayStation 3
...,...,...,...,...,...
95,Journey,92,8.3,21-7-2015,PlayStation 4
96,Bloodborne,92,8.8,24-3-2015,PlayStation 4
97,Eternal Darkness: Sanity's Requiem,92,8.7,23-6-2002,GameCube
98,Warcraft III: Reign of Chaos,92,9.2,3-7-2002,PC


In [ ]:
f.describe()